<a href="https://colab.research.google.com/github/jasonheesanglee/dangam/blob/main/development/dangam_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q torch==2.1.0
!pip install -q lingua-language-detector
!pip install -q transformers[torch] scipy datasets scikit-learn
!pip install -q nltk sentence-transformers spacy sentencepiece typing
!pip install -q konlpy jamo
!pip install -q emoji demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
import re
import gc
import csv
import math
import jamo
import nltk
import time
import emoji
import spacy
import pickle
import demoji
import random
import warnings
import numpy as np
import pandas as pd
import pkg_resources
from scipy import spatial
from tqdm.auto import tqdm
import torch.nn.functional as F
from nltk.corpus import stopwords
from transformers import BertModel
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from lingua import Language, LanguageDetectorBuilder
%cd '/content/drive/MyDrive/Colab Notebooks/Avocado_Land/jamo_con'
from han_util_unicode import join_jamos, split_syllables
%cd



/content/drive/MyDrive/Colab Notebooks/Avocado_Land/jamo_con
/root


In [4]:
lingua_languages = Language.all()
lingua_detection = LanguageDetectorBuilder.from_languages(*lingua_languages).build()

In [5]:
# kor_text_file = pd.read_csv('/content/My Drive/AvocadoLand/jamo_con/wordslistUnique.txt', encoding='utf-8')
# ko_base_dict = pd.read_csv(avo + '/jamo_con/ko_50k.txt')

tqdm.pandas()
warnings.filterwarnings('ignore')
eng_lemmatizer = WordNetLemmatizer()

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [7]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

config = {
          'ko_model_name': 'hun3359/klue-bert-base-sentiment',
          'ko_sub_model_name': 'WhitePeak/bert-base-cased-Korean-sentiment',
          'ko_word_senti_model_name': 'WhitePeak/bert-base-cased-Korean-sentiment',
          'en_model_name': 'SamLowe/roberta-base-go_emotions',
          'en_sub_model_name': 'citizenlab/twitter-xlm-roberta-base-sentiment-finetunned',
          'en_word_senti_model_name': 'citizenlab/twitter-xlm-roberta-base-sentiment-finetunned',
          'truncation': True,
          'padding': True,
          'max_length': 512,
          'alignment_threshold': 0.4,
          'emotion_threshold' : 0.3,
          'emotion_weight_reach_threshold':0.5,
          'emotion_weight_not_reach_threshold': 0.75,
          'specific_weight_reach_threshold': 0.1,
          'specific_weight_not_reach_threshold': 0.23,
          'noun_threshold' : 0.3,
        }
cfg = dotdict(config)

In [8]:
class DanGamConfig:
    """
    Configuration class for DanGam. It handles the setup of model parameters and preferences.

    Methods:
        cfg_info(): Prints the detailed information about configuration options.
        check_default(): Prints the default configuration settings.
    """
    VERSION = '0.0.135'
    CREATED_BY = 'jasonheesanglee\thttps://github.com/jasonheesanglee'

    def cfg_info(self) -> None:
        print(f"""
'ko_model_name' - The model that will run through the first loop of the sentence segmentation.
'ko_sub_model_name' - The model that will run through the second loop of the sentence segmentation.
'ko_word_senti_model_name' - The model that will through the loop of the word segmentation.
'text_col' - The name of the column that you want to segment the emotion.
'default_emotion_column' - Pre-labeled emotion by user.
'original_emotion_column' - Pre-segmented emotions by user.
\tPerforms the best if this section is segmented into 'positive', 'negative', 'neutral'.
\tUsed for accuracy evaluation.
'normalized_emotion_column' - Normalized pre-labeled emotion.
\tPerforms the best if this section is in English.
\tDirectly used from the second loop, since it will only segment positive, negative, neutral.
\tNot into 60 different emotions.
'sentence_emotion_column' - The column name of sentence emotion (pos/neg/neut) you want this module to set.
'sentence_specific_emotion_column' - The column name of sentence emotion (pos/neg/neut) you want this module to set.
'truncation' - Turning on and off Truncation throughout the module.
'max_length' - Max length for chunk_text
'emotion_threshold' - Threshold for emotion and specific emotion embeddings are adjusted accordingly to refine the combined embedding, ensuring a more nuanced sentiment analysis.
'alignment_threshold' - Threshold for the cosine similarity between the combined sentence-emotion embedding and each individual word embedding.
'emotion_weight_reach_threshold' - The weight to be multiplied on emotion embedding when similarity exceeds threshold.
'emotion_weight_not_reach_threshold' - The weight to be multiplied on emotion embedding when similarity doesn't exceed threshold.
'specific_weight_reach_threshold' - The weight to be multiplied on specific emotion embedding when similarity exceeds threshold.
'specific_weight_not_reach_threshold' - The weight to be multiplied on specific emotion embedding when similarity doesn't exceed threshold.
'noun_threshold' - Threshold for deciding the emotion_segment of a word.
        """)

    def check_default(self):
        return """
'ko_model_name': 'hun3359/klue-bert-base-sentiment',
\tBetter if you stick to this.
This is one of the only options that segments Korean sentences into 60 sentiment labels.
'ko_sub_model_name': 'WhitePeak/bert-base-cased-Korean-sentiment'
\tAny model that supports Positive/Negative segmentation will work.
'ko_word_senti_model_name': 'WhitePeak/bert-base-cased-Korean-sentiment'
\tAny model that supports Positive/Negative segmentation will work.
'text_col': 'text'
\tYou need to modify this if your col name is not 'text'.
'original_emotion_column': 'posneg'
\tYou need to modify this if your col name is not 'posneg'.
'default_emotion_column': 'default_emotion'
\tYou need to modify this if your col name is not 'default_emotion'.
'normalized_emotion_column': 'gpt_emotion'
\tYou need to modify this if your col name is not 'gpt_emotion'.
'sentence_emotion_column': 'klue_emo'
\tYou can leave it as it is, change it, as you wish.
'sentence_specific_emotion_column': 'klue_specific_emo'
\tYou can leave it as it is, change it, as you wish.
'truncation': True
'padding': True
'max_length': 512

### BELOW NEEDS CAUTION WHEN MODIFYING ###

'emotion_threshold' : 0.3
'alignment_threshold': 0.7
'emotion_weight_reach_threshold':0.5
'emotion_weight_not_reach_threshold': 0.75
'specific_weight_reach_threshold': 0.1
'specific_weight_not_reach_threshold': 0.23
'noun_threshold' : 0.3
"""


    def __init__(self, cfg=None):
        """
        Initializes the DanGamConfig with default or user-provided settings.

        Args:
            cfg (dict, optional): A dictionary containing configuration settings.
                                  If provided, it overrides the default settings.
        """
        config = {
            'ko_model_name': 'hun3359/klue-bert-base-sentiment',
            'ko_sub_model_name': 'WhitePeak/bert-base-cased-Korean-sentiment',
            'ko_word_senti_model_name': 'WhitePeak/bert-base-cased-Korean-sentiment',
            'en_model_name': 'SamLowe/roberta-base-go_emotions',
            'en_sub_model_name': 'citizenlab/twitter-xlm-roberta-base-sentiment-finetunned',
            'en_word_senti_model_name': 'citizenlab/twitter-xlm-roberta-base-sentiment-finetunned',
            'truncation': True,
            'padding': True,
            'max_length': 512,
            'alignment_threshold': 0.7,
            'emotion_threshold' : 0.3,
            'emotion_weight_reach_threshold':0.5,
            'emotion_weight_not_reach_threshold': 0.75,
            'specific_weight_reach_threshold': 0.1,
            'specific_weight_not_reach_threshold': 0.23,
            'noun_threshold' : 0.3,
        }

        if cfg is not None:
            if isinstance(cfg, dict):
                config.update(cfg)

        self.ko_model_name = config['ko_model_name']
        self.ko_sub_model_name = config['ko_sub_model_name']
        self.ko_word_senti_model_name = config['ko_word_senti_model_name']
        self.en_model_name = config['en_model_name']
        self.en_sub_model_name = config['en_sub_model_name']
        self.en_word_senti_model_name = config['en_word_senti_model_name']
        self.truncation = config['truncation']
        self.padding = config['padding']
        self.max_length = config['max_length']
        self.alignment_threshold = config['alignment_threshold']
        self.emotion_threshold = config['emotion_threshold']
        self.emotion_weight_reach_threshold = config['emotion_weight_reach_threshold']
        self.emotion_weight_not_reach_threshold = config['emotion_weight_not_reach_threshold']
        self.specific_weight_reach_threshold = config['specific_weight_reach_threshold']
        self.specific_weight_not_reach_threshold = config['specific_weight_not_reach_threshold']
        self.noun_threshold = config['noun_threshold']

    def get_config(self):
        """
        Returns the current configuration as a dictionary.
        """
        return {
            'ko_model_name': self.ko_model_name,
            'ko_sub_model_name': self.ko_sub_model_name,
            'ko_word_senti_model': self.ko_word_senti_model_name,
            'en_model_name': self.en_model_name,
            'en_sub_model_name': self.en_sub_model_name,
            'en_word_senti_model': self.en_word_senti_model_name,
            'truncation' : self.truncation,
            'padding': self.padding,
            'max_length': self.max_length,
            'alignment_threshold': self.alignment_threshold,
            'emotion_threshold': self.emotion_threshold,
            'emotion_weight_reach_threshold': self.emotion_weight_reach_threshold,
            'emotion_weight_not_reach_threshold': self.emotion_weight_not_reach_threshold,
            'specific_weight_reach_threshold': self.specific_weight_reach_threshold,
            'specific_weight_not_reach_threshold': self.specific_weight_not_reach_threshold,
            'noun_threshold': self.noun_threshold,

        }

In [9]:
import re

import torch
import numpy as np
from tqdm.auto import tqdm
import torch.nn.functional as F
from collections import defaultdict
# from dangam import DanGamConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel


class DotDict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


emotion_labels_ko = {
    0: "rage", 1: "whining", 2: "frustrated", 3: "irritated",
    4: "defensive", 5: "spiteful", 6: "restless", 7: "disgusted",
    8: "displeased", 9: "annoyed", 10: "sad", 11: "disappointed",
    12: "heartbreaking", 13: "regret", 14: "depressed", 15: "paralyzed",
    16: "pessimistic", 17: "tearful", 18: "dejected", 19: "disillusioned",
    20: "anxious", 21: "frightened", 22: "stressful", 23: "vulnerable",
    24: "confused", 25: "embarrassing", 26: "skeptical", 27: "worried",
    28: "cautious", 29: "nervous", 30: "wound", 31: "jealous",
    32: "betrayed", 33: "isolated", 34: "shocked", 35: "poor",
    36: "victimized", 37: "unfair", 38: "afflicted", 39: "abandoned",
    40: "bewildered", 41: "beleaguered", 42: "self conscious", 43: "lonely",
    44: "inferiority", 45: "guilty", 46: "ashamed", 47: "abominable",
    48: "pathetic", 49: "baffled", 50: "joyful", 51: "grateful",
    52: "trusting", 53: "comfortable", 54: "satisfying", 55: "thrilled",
    56: "relaxed", 57: "relieved", 58: "excited", 59: "confident"
}

emotion_labels_en_ko = {
    "rage": "분노", "whining": "툴툴대는", 'frustrated': "좌절한", 'irritated': "짜증내는",
    'defensive': "방어적인", 'spiteful': "악의적인", 'restless': "안달하는", 'disgusted': "구역질 나는",
    'displeased': "노여워하는", 'annoyed': "성가신", 'sad': "슬픔", 'disappointed': "실망한",
    'heartbreaking': "비통한", 'regret': "후회되는", 'depressed': "우울한", 'paralyzed': "마비된",
    'pessimistic': "염세적인", 'tearful': "눈물이 나는", 'dejected': "낙담한", 'disillusioned': "환멸을 느끼는",
    'anxious': "불안", 'frightened': "두려운", 'stressful': "스트레스 받는", 'vulnerable': "취약한",
    'confused': "혼란스러운", 'embarrassing': "당혹스러운", 'skeptical': "회의적인", 'worried': "걱정스러운",
    'cautious': "조심스러운", 'nervous': "초조한", 'wound': "상처", 'jealous': "질투하는",
    'betrayed': "배신당한", 'isolated': "고립된", 'shocked': "충격 받은", 'poor': "가난한 불우한",
    'victimized': "희생된", 'unfair': "억울한", 'afflicted': "괴로워하는", 'abandoned': "버려진",
    'bewildered': "당황", 'beleaguered': "고립된 당황한)", 'self conscious': "남의 시선을 의식하는", 'lonely': "외로운",
    'inferiority': "열등감", "guilty": "죄책감의", 'ashamed': "부끄러운", 'abominable': "혐오스러운",
    'pathetic': "한심한", 'baffled': "혼란스러운 (당황한)", 'joyful': "기쁨", 'grateful': "감사하는",
    'trusting': "신뢰하는", 'comfortable': "편안한", 'satisfying': "만족스러운", 'thrilled': "흥분",
    'relaxed': "느긋한", 'relieved': "안도하는", 'excited': "신이 난", 'confident': "자신하는"
}
neg_tag_ko = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
           10, 11, 12, 13, 14, 15, 16, 18, 19,
           20, 21, 22, 23, 24, 25, 26, 27,
           30, 32, 34, 35, 36, 37, 38, 39,
           40, 41, 43, 44, 45, 46, 47, 48, 49]
pos_tag_ko = [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
neut_tag_ko = [17, 28, 29, 31, 33, 42]

emotion_labels_en = {
    0: "admiration", 1: "amusement", 2: "anger", 3: "annoyance",
    4: "approval", 5: "caring", 6: "confusion", 7: "curiosity",
    8: "desire", 9: "disappointment", 10: "disapproval", 11: "disgust",
    12: "embarrassment", 13: "excitement", 14: "fear", 15: "gratitude",
    16: "grief", 17: "joy", 18: "love", 19: "nervousness",
    20: "optimism", 21: "pride", 22: "realization", 23: "relief",
    24: "remorse", 25: "sadness", 26: "surprise", 27: "neutral"
}
neg_tag_en = [2, 3, 6, 9, 10, 11, 12, 14, 16, 24, 25]
pos_tag_en = [0, 1, 4, 5, 8, 13, 15, 17, 18, 20, 21, 23, 26]
neut_tag_en = [7, 19, 22, 27]



class DanGam:
    """
    The DanGam class is designed for advanced emotion analysis in textual data. It utilizes
    sophisticated NLP models to accurately identify and categorize emotions at both sentence
    and word levels, providing nuanced insights into the emotional tone of text.

    Attributes:
        cfg (DotDict): Configuration settings for the DanGam instance.
        model (AutoModelForSequenceClassification): Primary model for general emotion classification.
        sub_model (AutoModelForSequenceClassification): Secondary model for more detailed emotion classification.
        word_senti_model (AutoModel): Model dedicated to word-level sentiment analysis.
        original_emotion_column, text_col, default_emotion_column, normalized_emotion_column, sentence_emotion_column, sentence_specific_emotion_column (str): Column names for various data attributes, configurable as per user needs.

    Methods:
        cfg_info(): Displays the current configuration settings of the DanGam instance.
        check_default(): Outputs the default configuration values for reference.
        check_config(): Returns the current configuration of DanGam as a dictionary.
        chunk_text(text, max_length): Splits a given text into manageable chunks for model processing.
        get_emotion(...): Analyzes and returns the overall emotion of a sentence based on model predictions.
        normalize_sentiment_scores(dissimilarities): Normalizes sentiment dissimilarity scores.
        word_emotions(sentence, emotion_label, specific_emotion_label): Segments words in a sentence and assigns them emotions.
        noun_emotions(sentence, noun_list): Analyzes and categorizes emotions associated with specific nouns in a sentence.

    Usage:
        - Initialize the class with default or custom configuration.
        - Use its methods to perform detailed emotion segmentation and analysis in textual content.
    """
    VERSION = '0.0.135'
    CREATED_BY = 'jasonheesanglee\thttps://github.com/jasonheesanglee'

    def __init__(self, cfg=None):
        """

    Args:
        cfg (dict, optional): A dictionary containing custom configuration settings. If provided, these settings will
                              override the default configuration. The configuration should specify details like model names,
                              tokenizer preferences, column names for data processing, and other relevant parameters.
                              If None, the default configuration is used.

    Attributes Initialization:
        - Initializes the model and tokenizer based on the provided or default configuration.
        - Sets up various class attributes like separator tokens and emotion labels.
        - Prepares lists of tags for categorizing emotions as positive, neutral, or negative.
    """
        print('''
CAUTION
This logic performs the best with the models that are pretrained with
AI HUB Dataset https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
or any Dataset that has 60 sentiment tags listed as described in https://huggingface.co/hun3359/klue-bert-base-sentiment/blob/main/config.json\n
You can also modify configuration by calling update_config()
        '''
              )
        if cfg is not None:
            self.cfg = DanGamConfig(cfg)
            # print('why?')
        else:
            self.cfg = DanGamConfig()
            # print('good')
        self.initialize_models()
        self.sep_token = ' >>>> '
        self.emotion_label_ko = emotion_labels_ko
        self.emotion_label_en = emotion_labels_en
        self.pos_tag_ko = pos_tag_ko
        self.neut_tag_ko = neut_tag_ko
        self.neg_tag_ko = neg_tag_ko
        self.pos_tag_en = pos_tag_en
        self.neut_tag_en = neut_tag_en
        self.neg_tag_en = neg_tag_en

    def config_info(self):
        """
        Prints the current configuration information of the DanGam.
        Includes details about the models used, text and emotion column names, and other settings.
        """
        print(f"""
'ko_model_name' - The model that will run through the first loop of the sentence segmentation.\n
'ko_sub_model_name' - The model that will run through the second loop of the sentence segmentation.\n
'ko_word_senti_model_name' - The model that will run through the second loop of the sentence segmentation.\n
'text_col' - The name of the column that you want to segment the emotion.\n
'default_emo_col' - Pre-labeled emotion by user.\n
'ori_emo_col' - Pre-segmented emotions by user.
\tPerforms the best if this section is segmented into 'positive', 'negative', 'neutral'.
\tUsed for accuracy evaluation.\n
'normalized_emo_col' - Normalized pre-labeled emotion.
\tPerforms the best if this section is in English.
\tDirectly used from the second loop, since it will only segment positive, negative, neutral.
\tNot into 60 different emotions.\n
'truncation' - Bool : Turning on and off Truncation throughout the module.\n
'sent_emo_col' - The column name of sentence emotion (pos/neg/neut) you want this module to set.\n
'sent_spec_emo_col' - The column name of sentence emotion (pos/neg/neut) you want this module to set.\n
'max_length' - Max length for chunk_text.
        """)

    def initialize_models(self):
        """
        Initialize models based on the current configuration.
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        ### Korean

        self.ko_tokenizer = AutoTokenizer.from_pretrained(self.cfg.ko_model_name)
        self.ko_model = AutoModelForSequenceClassification.from_pretrained(self.cfg.ko_model_name)
        self.ko_model.to(self.device)

        self.ko_sub_tokenizer = AutoTokenizer.from_pretrained(self.cfg.ko_sub_model_name)
        self.ko_sub_model = AutoModelForSequenceClassification.from_pretrained(self.cfg.ko_sub_model_name)
        self.ko_sub_model.to(self.device)

        self.ko_word_senti_tokenizer = AutoTokenizer.from_pretrained(self.cfg.ko_word_senti_model_name)
        self.ko_word_senti_model = AutoModel.from_pretrained(self.cfg.ko_word_senti_model_name)
        self.ko_word_senti_model.to(self.device)

        ### English

        self.en_tokenizer = AutoTokenizer.from_pretrained(self.cfg.en_model_name)
        self.en_model = AutoModelForSequenceClassification.from_pretrained(self.cfg.en_model_name)
        self.en_model.to(self.device)

        self.en_sub_tokenizer = AutoTokenizer.from_pretrained(self.cfg.en_sub_model_name)
        self.en_sub_model = AutoModelForSequenceClassification.from_pretrained(self.cfg.en_sub_model_name)
        self.en_sub_model.to(self.device)

        self.en_word_senti_tokenizer = AutoTokenizer.from_pretrained(self.cfg.en_word_senti_model_name)
        self.en_word_senti_model = AutoModel.from_pretrained(self.cfg.en_word_senti_model_name)
        self.en_word_senti_model.to(self.device)


        ### model configurations

        self.truncation = self.cfg.truncation
        self.padding = self.cfg.padding
        self.max_length = self.cfg.max_length
        self.align_th = self.cfg.alignment_threshold
        self.emo_th = self.cfg.emotion_threshold
        self.emo_wt_reach_th = self.cfg.emotion_weight_reach_threshold
        self.emo_wt_n_reach_th = self.cfg.emotion_weight_not_reach_threshold
        self.spec_wt_reach_th = self.cfg.specific_weight_reach_threshold
        self.spec_wt_n_reach_th = self.cfg.specific_weight_not_reach_threshold
        self.noun_th = self.cfg.noun_threshold

    def lang_conf_rate(self, text):
        conf_rate = {}
        try:
            for word in text.split():
                possible_lang = lingua_detection.compute_language_confidence_values(word)
                for confidence in possible_lang:
                    try:
                        conf_rate[confidence.language.name] += confidence.value
                    except KeyError:
                        conf_rate[confidence.language.name] = confidence.value

            conf_rate_per = {}
            for key, value in conf_rate.items():
                if sum(conf_rate.values()) != 0:
                    conf_rate_per[key] = value / sum(conf_rate.values())
                else:
                    conf_rate_per[key] = 0
            return conf_rate_per
        except:
            pass

    def lang_detector(self, text):
        text_lang = None
        try:
            lang_ratio = self.lang_conf_rate(text)

            max_rate = 0
            lang_temp = {}
            for lang, rate in lang_ratio.items():
                if rate > max_rate:
                    max_rate = rate
                    text_lang = lang
                    lang_temp = {lang: rate}
                elif rate == max_rate:
                    lang_temp[lang] = rate
            if 'JAPANESE' in lang_temp and 'CHINESE' in lang_temp and lang_temp['JAPANESE'] == lang_temp['CHINESE']:
                text_lang = 'JAPANESE'
            return text_lang
        except:
            return 'unidentified'

    def update_config(self, new_config):
        """
        Update the configuration of DanGam and reinitialize components as necessary.

        Args:
            new_cfg (dict): A dictionary containing the new configuration settings.
        """
        self.cfg = DanGamConfig(new_config)
        self.initialize_models()

    def check_default(self):
        print(self.cfg.check_default())

    def check_config(self):
        return self.cfg.get_config()

    def chunk_text(self, text: str, max_length=512, language='KOREAN'):
        """
        Splits a given text into chunks for processing.
        Ensures that each chunk is within the specified maximum length.

        Args:
            text (str): The text to be chunked.
            max_length (int): Maximum length of each text chunk.

        Yields:
            str: Chunks of the original text.
        """
        if language == 'KOREAN':
            tokens = self.ko_tokenizer.tokenize(text)

        elif language == 'ENGLISH':
            tokens = self.en_tokenizer.tokenize(text)


        chunk_size = max_length - 2
        for i in range(0, len(tokens), chunk_size):
            yield ' '.join(tokens[i:i + chunk_size])

    def get_emotion(self,
                sentence: str,
                original_emotion: str = None,
                default_specific_emotion: str = None,
                normalized_emotion: str = None,
                language: str = None,
                ):
        """
        Determines the overall emotion of a given sentence by analyzing it in chunks.
        Considers both the general and specific emotions to enhance accuracy.

        Args:
            sentence : The sentence to extract emotion from.
            original_emotion (str) -> optional : The pre-segmented emotion (positive, negative, neutral)
            default_specific_emotion (str) -> optional : The pre-segmented specific emotion (love, thrilled, happy, sad, etc..)
            normalized_emotion (str) -> optional : Normalized User input emotion (good food, bad person, lovely day, etc..)

        Returns:
            emotion (str) : string of overall emotion of the sentence. (positive, neutral, negative)
            specific_emotion (str) : string of specific emotion of the sentence. (one out of 60 emotions)
        """
        pattern = '[^a-zA-Zㄱ-ㅎㅏ-ㅢ가-힣0-9+]'
        sentence = re.sub(pattern, ' ', sentence)
        # if language == None:
        #     language = self.lang_detector(sentence)
        if original_emotion != None or default_specific_emotion != None or normalized_emotion != None:
            chunks = list(self.chunk_text(sentence, language=language))
            sum_prob = None
            num_chunks = 0

            for chunk in chunks:
                if language == 'KOREAN':
                    if normalized_emotion != '-':
                        inputs = self.ko_tokenizer(normalized_emotion + self.sep_token + chunk, return_tensors='pt',
                                                padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    else:
                        inputs = self.ko_tokenizer(default_specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                                padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}
                    with torch.no_grad():
                        logits = self.ko_model(**inputs).logits

                elif language == 'ENGLISH':
                    if normalized_emotion != '-':
                        inputs = self.en_tokenizer(normalized_emotion + self.sep_token + chunk, return_tensors='pt',
                                                padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    else:
                        inputs = self.en_tokenizer(default_specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                                padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}
                    with torch.no_grad():
                        logits = self.en_model(**inputs).logits

                probabilities = torch.nn.functional.softmax(logits, dim=-1)

                if sum_prob is None:
                    sum_prob = probabilities
                else:
                    sum_prob += probabilities
                num_chunks += 1

            if num_chunks != 0 and sum_prob is not None:
                avg_prob = sum_prob / num_chunks
                sentiment_idx = avg_prob.argmax().item()
            else:
                avg_prob = 0
                sentiment_idx = avg_prob

            if language == 'KOREAN':
                if sentiment_idx in self.neg_tag_ko:
                    emotion = 'negative'
                    specific_emotion = self.emotion_label_ko[sentiment_idx]
                elif sentiment_idx in self.pos_tag_ko:
                    emotion = 'positive'
                    specific_emotion = self.emotion_label_ko[sentiment_idx]
                else:
                    emotion = 'neutral'
                    specific_emotion = self.emotion_label_ko[sentiment_idx]

            elif language == 'ENGLISH':
                if sentiment_idx in self.neg_tag_en:
                    emotion = 'negative'
                    specific_emotion = self.emotion_label_en[sentiment_idx]
                elif sentiment_idx in self.pos_tag_en:
                    emotion = 'positive'
                    specific_emotion = self.emotion_label_en[sentiment_idx]
                else:
                    emotion = 'neutral'
                    specific_emotion = self.emotion_label_en[sentiment_idx]


                if original_emotion.strip() == emotion.strip():
                    return emotion, specific_emotion
                else:
                    emotion_counts = {'positive': 0, 'negative': 0}


                if original_emotion.strip() == emotion.strip():
                    return emotion, specific_emotion
                else:
                    emotion_counts = {'positive': 0, 'negative': 0, 'neutral':0}

                for chunk in chunks:
                    if language == 'KOREAN':
                        if normalized_emotion != '-':
                            inputs = self.ko_sub_tokenizer(normalized_emotion + self.sep_token + chunk, return_tensors='pt',
                                                        padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                        else:
                            inputs = self.ko_sub_tokenizer(default_specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                                        padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                        inputs = {k: v.to(self.device) for k, v in inputs.items()}

                        with torch.no_grad():
                            logits = self.ko_sub_model(**inputs).logits

                    elif language == 'ENGLISH':
                        if normalized_emotion != '-':
                            inputs = self.en_sub_tokenizer(normalized_emotion + self.sep_token + chunk, return_tensors='pt',
                                                        padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                        else:
                            inputs = self.en_sub_tokenizer(default_specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                                        padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                        inputs = {k: v.to(self.device) for k, v in inputs.items()}

                        with torch.no_grad():
                            logits = self.en_sub_model(**inputs).logits

                    probabilities = torch.nn.functional.softmax(logits, dim=-1)
                    emotion_pred = probabilities.argmax().item()  ##### majority votes

                    if emotion_pred == 0:
                        emotion_counts['negative'] += 1
                    elif emotion_pred == 1:
                        emotion_counts['positive'] += 1
                    else:
                        emotion_counts['neutral'] += 1

                if emotion_counts['neutral'] > emotion_counts['positive'] + emotion_counts['negative']:
                    emotion = 'neutral'
                elif emotion_counts['negative'] > emotion_counts['positive']:
                    emotion = 'negative'
                elif emotion_counts['negative'] < emotion_counts['positive']:
                    emotion = 'positive'
                else:
                    emotion = 'neutral'

                specific_emotion = normalized_emotion
                if original_emotion.strip() == emotion.strip():
                    return emotion, specific_emotion
                else:  ############ 여기에 아예 catboost를 써볼까....??
                    sum_prob = None
                    num_chunks = 0
                    for chunk in chunks:
                        if language =='KOREAN':
                            if normalized_emotion != '-':
                                inputs = self.ko_sub_tokenizer(normalized_emotion + self.sep_token + chunk, return_tensors='pt',
                                                            padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                            else:
                                inputs = self.ko_sub_tokenizer(default_specific_emotion + self.sep_token + chunk,
                                                            return_tensors='pt', padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                            inputs = {k: v.to(self.device) for k, v in inputs.items()}

                            with torch.no_grad():
                                logits = self.ko_sub_model(**inputs).logits

                        elif language == 'ENGLISH':
                            if normalized_emotion != '-':
                                inputs = self.en_sub_tokenizer(normalized_emotion + self.sep_token + chunk, return_tensors='pt',
                                                            padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                            else:
                                inputs = self.en_sub_tokenizer(default_specific_emotion + self.sep_token + chunk,
                                                            return_tensors='pt', padding=self.padding,
                                                            truncation=self.truncation, max_length=self.max_length)

                            inputs = {k: v.to(self.device) for k, v in inputs.items()}

                            with torch.no_grad():
                                logits = self.en_sub_model(**inputs).logits

                        probabilities = torch.nn.functional.softmax(logits, dim=-1)
                        if sum_prob is None:
                            sum_prob = probabilities
                        else:
                            sum_prob += probabilities
                        num_chunks += 1

                    if num_chunks != 0 and sum_prob is not None:
                        avg_prob = sum_prob / num_chunks
                        emotion_ = avg_prob.argmax().item()

                    else:
                        emotion_ = None

                    if emotion_ == 0:
                        emotion = 'negative'

                    elif emotion_ == 1:
                        emotion = 'positive'
                    else:
                        emotion = 'neutral'
                    return emotion, specific_emotion

        else:
            chunks = list(self.chunk_text(sentence, max_length=int(self.max_length/5),language=language))
            sum_prob = None
            num_chunks = 0

            for chunk_no in range(len(chunks)):
                if language == 'KOREAN':
                    if chunk_no != len(chunks)-1:
                        inputs = self.ko_tokenizer(chunks[chunk_no] + self.sep_token + chunks[chunk_no+1], return_tensors='pt',
                                                padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    else:
                        inputs = self.ko_tokenizer(chunks[chunk_no] + self.sep_token, return_tensors='pt',
                            padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    with torch.no_grad():
                        logits = self.ko_model(**inputs).logits

                elif language =='ENGLISH':
                    if chunk_no != len(chunks)-1:
                        inputs = self.en_tokenizer(chunks[chunk_no] + self.sep_token + chunks[chunk_no+1], return_tensors='pt',
                                                padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    else:
                        inputs = self.en_tokenizer(chunks[chunk_no] + self.sep_token, return_tensors='pt',
                            padding=self.padding, truncation=self.truncation, max_length=self.max_length)
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    with torch.no_grad():
                        logits = self.en_model(**inputs).logits

                probabilities = torch.nn.functional.softmax(logits, dim=-1)

                if sum_prob is None:
                    sum_prob = probabilities
                else:
                    sum_prob += probabilities
                num_chunks += 1

            if num_chunks != 0 and sum_prob is not None:
                avg_prob = sum_prob / num_chunks
                sentiment_idx = avg_prob.argmax().item()
            else:
                avg_prob = 0
                sentiment_idx = avg_prob
            if language == 'KOREAN':
                if sentiment_idx in self.neg_tag_ko:
                    emotion = 'negative'
                    specific_emotion = self.emotion_label_ko[sentiment_idx]
                elif sentiment_idx in self.pos_tag_ko:
                    emotion = 'positive'
                    specific_emotion = self.emotion_label_ko[sentiment_idx]
                else:
                    emotion = 'neutral'
                    specific_emotion = self.emotion_label_ko[sentiment_idx]

            elif language == 'ENGLISH':
                if sentiment_idx in self.neg_tag_en:
                    emotion = 'negative'
                    specific_emotion = self.emotion_label_en[sentiment_idx]
                elif sentiment_idx in self.pos_tag_en:
                    emotion = 'positive'
                    specific_emotion = self.emotion_label_en[sentiment_idx]
                else:
                    emotion = 'neutral'
                    specific_emotion = self.emotion_label_en[sentiment_idx]

            sum_prob = None
            num_chunks = 0

            for chunk in chunks:
                if language == 'KOREAN':
                    inputs = self.ko_tokenizer(emotion + self.sep_token + specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                            padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    with torch.no_grad():
                        logits = self.ko_model(**inputs).logits


                elif language == 'ENGLISH':
                    inputs = self.en_tokenizer(emotion + self.sep_token + specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                            padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    with torch.no_grad():
                        logits = self.en_model(**inputs).logits

                probabilities = torch.nn.functional.softmax(logits, dim=-1)

                if sum_prob is None:
                    sum_prob = probabilities
                else:
                    sum_prob += probabilities
                num_chunks += 1

            if num_chunks != 0 and sum_prob is not None:
                avg_prob = sum_prob / num_chunks
                sentiment_idx = avg_prob.argmax().item()
            else:
                avg_prob = 0
                sentiment_idx = avg_prob
            if language == 'KOREAN':
                if sentiment_idx in self.neg_tag_ko:
                    new_emotion = 'negative'
                    new_specific_emotion = self.emotion_label_ko[sentiment_idx]
                elif sentiment_idx in self.pos_tag_ko:
                    new_emotion = 'positive'
                    new_specific_emotion = self.emotion_label_ko[sentiment_idx]
                else:
                    new_emotion = 'neutral'
                    new_specific_emotion = self.emotion_label_ko[sentiment_idx]

            elif language == 'ENGLISH':
                if sentiment_idx in self.neg_tag_en:
                    new_emotion = 'negative'
                    new_specific_emotion = self.emotion_label_en[sentiment_idx]
                elif sentiment_idx in self.pos_tag_en:
                    new_emotion = 'positive'
                    new_specific_emotion = self.emotion_label_en[sentiment_idx]
                else:
                    new_emotion = 'neutral'
                    new_specific_emotion = self.emotion_label_en[sentiment_idx]

            if new_emotion.strip() == emotion.strip():
                return new_emotion, new_specific_emotion
            else:
                # return emotion, specific_emotion
                emotion_counts = {'positive': 0, 'negative': 0, 'neutral': 0}

                for chunk in chunks:
                    if language == 'KOREAN':
                        inputs = self.ko_sub_tokenizer(emotion + self.sep_token + specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                                    padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                        inputs = {k: v.to(self.device) for k, v in inputs.items()}

                        with torch.no_grad():
                            logits = self.ko_sub_model(**inputs).logits

                    if language == 'ENGLISH':
                        inputs = self.en_sub_tokenizer(emotion + self.sep_token + specific_emotion + self.sep_token + chunk, return_tensors='pt',
                                                    padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                        inputs = {k: v.to(self.device) for k, v in inputs.items()}

                        with torch.no_grad():
                            logits = self.en_sub_model(**inputs).logits

                    probabilities = torch.nn.functional.softmax(logits, dim=-1)
                    emotion_pred = probabilities.argmax().item()  ##### majority votes
                    if emotion_pred == 0:
                        emotion_counts['negative'] += 1
                    elif emotion_pred == 1:
                        emotion_counts['positive'] += 1
                    else:
                        emotion_counts['neutral'] += 1

                if emotion_counts['neutral'] > emotion_counts['positive'] + emotion_counts['negative']:
                    new_emotion = 'neutral'
                elif emotion_counts['negative'] > emotion_counts['positive']:
                    new_emotion = 'negative'
                elif emotion_counts['negative'] < emotion_counts['positive']:
                    new_emotion = 'positive'
                else:
                    new_emotion = 'neutral'

                if new_emotion.strip() == emotion.strip():
                    return new_emotion, new_specific_emotion
                else:  ############ 여기에 아예 catboost를 써볼까....??
                    sum_prob = None
                    num_chunks = 0
                    for chunk in chunks:
                        if language =='KOREAN':
                            inputs = self.ko_sub_tokenizer(emotion + self.sep_token + specific_emotion + self.sep_token + chunk,
                                                        return_tensors='pt',
                                                        padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                            inputs = {k: v.to(self.device) for k, v in inputs.items()}

                            with torch.no_grad():
                                logits = self.ko_sub_model(**inputs).logits

                        if language =='ENGLISH':
                            inputs = self.en_sub_tokenizer(emotion + self.sep_token + specific_emotion + self.sep_token + chunk,
                                                        return_tensors='pt',
                                                        padding=self.padding, truncation=self.truncation, max_length=self.max_length)

                            inputs = {k: v.to(self.device) for k, v in inputs.items()}

                            with torch.no_grad():
                                logits = self.en_sub_model(**inputs).logits

                        probabilities = torch.nn.functional.softmax(logits, dim=-1)
                        if sum_prob is None:
                            sum_prob = probabilities
                        else:
                            sum_prob += probabilities
                        num_chunks += 1

                    if num_chunks != 0 and sum_prob is not None:
                        avg_prob = sum_prob / num_chunks
                        emotion_ = avg_prob.argmax().item()

                    else:
                        emotion_ = None

                    if emotion_ == 0:
                        new_emotion = 'negative'

                    elif emotion_ == 1:
                        new_emotion = 'positive'
                    else:
                        new_emotion = 'neutral'

                    return new_emotion, new_specific_emotion

    def get_word_embeddings(self, sentence, language):
        """
        Retrieves word embeddings for a given sentence using the specified tokenizer and model.

        Args:
            sentence (str): The sentence for which to get word embeddings.
            max_length (int): Maximum length for tokenization.

        Returns:
            tuple: A tuple containing word embeddings and offset mappings.
        """
        # pattern = '[^a-zA-Zㄱ-ㅎㅏ-ㅢ가-힣0-9+]'
        # sentence = re.sub(pattern, ' ', sentence)
        if language =='KOREAN':
            inputs = self.ko_word_senti_tokenizer(sentence, return_tensors="pt", max_length=self.max_length,
                                                  truncation=self.truncation, padding='max_length',
                                                  return_offsets_mapping=True)
        if language =='ENGLISH':
            print(f'here here here : {sentence}')
            inputs = self.en_word_senti_tokenizer(sentence, return_tensors="pt", max_length=self.max_length,
                                                  truncation=self.truncation, padding='max_length',
                                                  return_offsets_mapping=True) #, is_split_into_words=True)#, return_special_tokens_mask=False)
        offset_mapping = inputs.pop('offset_mapping')
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        print(f'inputs \n{inputs}\n')
        if language == 'KOREAN':
            with torch.no_grad():
                outputs = self.ko_word_senti_model(**inputs)
        if language == 'ENGLISH':
            with torch.no_grad():
                outputs = self.en_word_senti_model(**inputs)
        print(f'outputs \n{outputs}\n')
        embeddings = outputs.last_hidden_state
        print(f'embeddings\n{embeddings}\n')
        print(f'offset_mapping\n{offset_mapping}\n')
        return embeddings, offset_mapping

    def get_sentence_embedding(self, sentence, language):
        """
        Computes the embedding of a sentence by averaging the embeddings of its constituent tokens.

        Args:
            sentence (str): The sentence to compute embedding for.
            max_length (int): Maximum length for tokenization.

        Returns:
            torch.Tensor: The computed sentence embedding.
        """
        # pattern = '[^a-zA-Zㄱ-ㅎㅏ-ㅢ가-힣0-9+]'
        # sentence = re.sub(pattern, ' ', sentence)
        if language =='KOREAN':
            inputs = self.ko_word_senti_tokenizer(sentence, return_tensors="pt", max_length=self.max_length,
                                                  truncation=self.truncation,
                                                  padding="max_length")
        if language =='ENGLISH':
            inputs = self.en_word_senti_tokenizer(sentence, return_tensors="pt", max_length=self.max_length,
                                                  truncation=self.truncation,
                                                  padding="max_length")

        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        if language == 'KOREAN':
            with torch.no_grad():
                outputs = self.ko_word_senti_model(**inputs)
        if language == 'ENGLISH':
            with torch.no_grad():
                outputs = self.en_word_senti_model(**inputs)

        # Average the token embeddings to get the sentence embedding
        sentence_embedding = outputs.last_hidden_state[:, 0, :]
        return sentence_embedding

    def get_emotion_embedding(self, emotion, language):
        """
        Computes the embedding for a general emotion label.

        Args:
            emotion (str): The emotion label for which to compute the embedding.

        Returns:
            torch.Tensor: The embedding of the specified emotion.
        """
        # pattern = '[^a-zA-Zㄱ-ㅎㅏ-ㅢ가-힣0-9+]'
        # sentence = re.sub(pattern, ' ', emotion)
        if language =='KOREAN':
            inputs = self.ko_word_senti_tokenizer(emotion, return_tensors='pt', padding=self.padding,
                                                  truncation=self.truncation, max_length=self.max_length)
        elif language =='ENGLISH':
            inputs = self.en_word_senti_tokenizer(emotion, return_tensors='pt', padding=self.padding,
                                                  truncation=self.truncation, max_length=self.max_length)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        if language == 'KOREAN':
            with torch.no_grad():
                outputs = self.ko_word_senti_model(**inputs)

        if language == 'ENGLISH':
            with torch.no_grad():
                outputs = self.en_word_senti_model(**inputs)

        return outputs.last_hidden_state.mean(dim=1)

    def get_specific_emotion_embedding(self, specific_emotion, language):
        """
        Computes the embedding for a specific emotion label.

        Args:
            specific_emotion (str): The specific emotion label for which to compute the embedding.

        Returns:
            torch.Tensor: The embedding of the specified specific emotion.
        """
        # pattern = '[^a-zA-Zㄱ-ㅎㅏ-ㅢ가-힣0-9+]'
        # sentence = re.sub(pattern, ' ', specific_emotion)
        if language == 'KOREAN':
            inputs = self.ko_word_senti_tokenizer(specific_emotion, return_tensors='pt', padding=self.padding,
                                                 truncation=self.truncation, max_length=self.max_length)
        if language == 'ENGLISH':
            inputs = self.en_word_senti_tokenizer(specific_emotion, return_tensors='pt', padding=self.padding,
                                                 truncation=self.truncation, max_length=self.max_length)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        if language =='KOREAN':
            with torch.no_grad():
                outputs = self.ko_word_senti_model(**inputs)

        if language =='ENGLISH':
            with torch.no_grad():
                outputs = self.en_word_senti_model(**inputs)

        return outputs.last_hidden_state.mean(dim=1)

    def calculate_vector_differences(self, sentence, emotion, specific_emotion, language):
        """
        Calculates vector differences (dissimilarities) between words in a sentence and a combined emotion embedding.

        Args:
            sentence (str): The sentence for analysis.
            emotion (str): General emotion label for the sentence.
            specific_emotion (str): Specific emotion label for the sentence.

        Returns:
            list: A list of dissimilarity scores for each word in the sentence.
        """
        pattern = '[^a-zA-Zㄱ-ㅎㅏ-ㅢ가-힣0-9+]'
        sentence = re.sub(pattern, ' ', sentence)
        word_embeddings, offset_mapping = self.get_word_embeddings(sentence, language)
        sentence_embedding = self.get_sentence_embedding(sentence, language)
        emotion_embedding = self.get_emotion_embedding(emotion, language)
        specific_emotion_embedding = self.get_emotion_embedding(specific_emotion, language)

        general_alignment = F.cosine_similarity(sentence_embedding, emotion_embedding, dim=1)
        specific_alignment = F.cosine_similarity(sentence_embedding, specific_emotion_embedding, dim=1)

        emotion_weight = self.emo_wt_reach_th if general_alignment.item() > self.emo_th else self.emo_wt_n_reach_th
        specific_weight = self.spec_wt_reach_th if specific_alignment.item() > self.emo_th else self.spec_wt_n_reach_th
        sentence_weight = 1 - (emotion_weight + specific_weight)

        combined_embedding = (emotion_embedding * emotion_weight -
                              sentence_embedding * sentence_weight +
                              specific_emotion_embedding * specific_weight) / 2

        # combined_embedding = (sentence_embedding - emotion_embedding) / 2

        dissimilarities = []

        for word_embedding in word_embeddings.squeeze():
            sim = F.cosine_similarity(combined_embedding, word_embedding.unsqueeze(0))
            if sim > self.align_th:
                adjust_sim = sim.item()
            else:
                adjust_sim = 1 - sim.item()

            dissimilarities.append(adjust_sim)

        return dissimilarities

    def normalize_sentiment_scores(self, dissimilarities):
        """
        Normalizes sentiment dissimilarity scores to a standard range for consistency.

        Args:
            dissimilarities (list): A list of dissimilarity scores.

        Returns:
            list: Normalized sentiment scores.
        """
        scores = np.array(dissimilarities)
        mean = np.mean(scores)
        std = np.std(scores)
        if std != 0:
            normalized_scores = (scores - mean) / std
        else:
            normalized_scores = scores - mean

        # Clip scores to be within -1 and 1
        normalized_scores = np.clip(normalized_scores, -1, 1)
        return normalized_scores.tolist()

    def assign_word_sentiment_scores(self, sentence, normalized_scores, language):
        """
        Assigns sentiment scores to each word in a sentence based on normalized dissimilarities.

        Args:
            sentence (str): The sentence to assign word sentiment scores.
            normalized_scores (list): Normalized sentiment scores for each token in the sentence.

        Returns:
            dict: A dictionary mapping each word in the sentence to its sentiment score.
        """
        if language =='KOREAN':
            encoded = self.ko_word_senti_tokenizer.encode_plus(sentence, return_tensors='pt', truncation=self.truncation,
                                                            padding='max_length', max_length=512)
            tokenized_words = self.ko_word_senti_tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
        elif language =='ENGLISH':
            encoded = self.en_word_senti_tokenizer.encode_plus(sentence, return_tensors='pt', truncation=self.truncation,
                                                            padding='max_length', max_length=512)
            tokenized_words = self.en_word_senti_tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])

        word_scores = []
        current_word = ""
        current_score = 0
        num_tokens = 0
        for token, score in zip(tokenized_words, normalized_scores):

            if token.startswith("##"):
                current_word += token.lstrip("##")
                current_score += score
                num_tokens += 1
            elif token.startswith('▁'):
                current_word += token.lstrip("▁")
                current_score += score
                num_tokens += 1
            else:
                if language =='KOREAN' and current_word and not current_word.startswith('['):
                    average_score = current_score / num_tokens if num_tokens > 0 else current_score
                    word_scores.append({current_word: average_score})
                elif language == 'ENGLISH' and current_word and not current_word.startswith('<'):
                    average_score = current_score / num_tokens if num_tokens > 0 else current_score
                    word_scores.append({current_word: average_score})
                current_word = token
                current_score = score
                num_tokens = 1

        # Add the last word
        if current_word and not current_word.startswith('['):
            average_score = current_score / num_tokens if num_tokens > 0 else current_score
            word_scores.append({current_word: average_score})

        return word_scores

    def word_emotions(self, sentence: str, emotion: str = None, specific_emotion: str = None, language=None):
        """
        Segments a sentence and assigns emotions to each word based on the overall sentence emotion and specific emotion.

        Args:
            sentence (str): The sentence for segmentation.
            emotion (str) -> Optional: The general emotion of the sentence.
            specific_emotion (str) -> Optional: The specific emotion of the sentence.

        Returns:
            dict: A dictionary mapping each word in the sentence to its assigned emotion.
        """

        pattern = '[^ㄱ-ㅣ가-힣a-zA-Z0-9+]'
        sentence = re.sub(pattern, ' ', sentence)
        if emotion != None:
            emotion = re.sub(pattern, ' ', emotion)
        if specific_emotion != None:
            specific_emotion = re.sub(pattern, ' ', specific_emotion)
        if language == None:
            language = self.lang_detector(sentence)
        if emotion == None or specific_emotion == None:
              emotion, specific_emotion = self.get_emotion(sentence, language=language)
        ################################### 여기까지 ok #################################################
        dissimilarities = self.calculate_vector_differences(sentence, emotion, specific_emotion, language=language)

        norm_senti_score = self.normalize_sentiment_scores(dissimilarities)
        word_sentiment_scores = self.assign_word_sentiment_scores(sentence, norm_senti_score, language=language)
        return word_sentiment_scores

    def noun_emotions(self,
                      sentence: str,
                      noun_list: list,
                      emotion : str = None,
                      specific_emotion : str = None,
                      count: bool=False,
                      language: str=None):
        """
        Analyzes emotions associated with specific nouns within a sentence.

        Args:
            sentence (str): The sentence containing the nouns for emotion analysis.
            emotion (str) -> Optional: The general emotion of the sentence.
            specific_emotion (str) -> Optional: The specific emotion of the sentence.
            noun_list (list): A list of nouns to analyze within the sentence.
            count (bool) : True or False for switching on off counting the number of nouns in each segment.
        Returns:
            dict: A dictionary categorizing nouns into positive, neutral, and negative based on their associated emotions.
        """
        language = self.lang_detector(sentence)
        if emotion == None or specific_emotion == None:
            emotion, specific_emotion = self.get_emotion(sentence, language=language)
        word_emo_list = self.word_emotions(sentence, emotion, specific_emotion, language)

        pos = defaultdict(list)
        neut = defaultdict(list)
        neg = defaultdict(list)

        pos_count = defaultdict(int)
        neut_count = defaultdict(int)
        neg_count = defaultdict(int)

        for noun in noun_list:
            for dict_ in word_emo_list:
                for word, score in dict_.items():
                    if noun in word:
                        if score > self.noun_th:  # when positive
                            pos[noun].append(score)
                            pos_count[noun] += 1
                        elif score < -1 * self.noun_th:
                            neg[noun].append(score)
                            neg_count[noun] += 1
                        else:
                            neut[noun].append(score)
                            neut_count[noun] += 1

        positive_noun_score = {}
        neutral_noun_score = {}
        negative_noun_score = {}

        for word, score_list in dict(pos).items():
            avg_score = sum(score_list) / len(score_list)
            positive_noun_score[word] = avg_score
        for word, score_list in dict(neut).items():
            avg_score = sum(score_list) / len(score_list)
            neutral_noun_score[word] = avg_score
        for word, score_list in dict(neg).items():
            avg_score = sum(score_list) / len(score_list)
            negative_noun_score[word] = avg_score

        scores = [dict(positive_noun_score), dict(neutral_noun_score), dict(negative_noun_score)]
        counts = [dict(pos_count), dict(neut_count), dict(neg_count)]

        if count == True:
            return scores, counts
        else:
            return scores

In [10]:
dangam = DanGam()


CAUTION
This logic performs the best with the models that are pretrained with
AI HUB Dataset https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
or any Dataset that has 60 sentiment tags listed as described in https://huggingface.co/hun3359/klue-bert-base-sentiment/blob/main/config.json

You can also modify configuration by calling update_config()
        


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at citizenlab/twitter-xlm-roberta-base-sentiment-finetunned and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
text = "나는 방금 먹은 마라탕이 너무 좋다. 적당한 양념에 알싸한 마라향이 미쳤다. 그런데 고수는 진짜 싫다!"
# dangam.lang_detector(text)
text_en = "I love eating pizza. Mushrooms are amazing, but I do not know why pineapple should go on top of pizza."
# emotion, specified_emotion = dangam.get_emotion(text, language=dangam.lang_detector(text))
# print(emotion, specified_emotion)

print(dangam.word_emotions(text))
print()
print(dangam.noun_emotions(text, ['마라탕', '양념', '마라향', '고수']))
print()
print()
# print(dangam.get_emotion(text_en))
print()
print(dangam.word_emotions(text_en))
print()
print(dangam.noun_emotions(text_en, ['pizza', 'food', 'mushrooms', 'pinapple', 'pizza']))



inputs 
{'input_ids': tensor([[   101, 100585,   9328,  40032,   9266,  10892,   9246,  17342, 119341,
          10739,   9004,  32537,   9685,  11903,   9664, 103088,   9543, 118739,
          10530,   9524, 119089,  11102,   9246,  17342,  79544,  10739,   9309,
          90821,   8924,  56710,  28911,   8888,  93835,   9708, 119235,   9490,
          11903,    102,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
import os
import smtplib
from smtplib import SMTPException
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

gmail_user = 'volvstang@gmail.com'
gmail_pw = 'nkyn buog yubo tchf' ############# deleted this for privacy

mail_from = gmail_user
mail_to = 'jason.heesang.lee96@gmail.com'

mail_subject = f'sentence_emotion_231213_5pm.csv'
mail_message_body = f'Check the attachments'

msg = MIMEMultipart()
msg['Subject'] = mail_subject
msg['From'] = mail_from
msg['To'] = mail_to

msg.attach(MIMEText(mail_message_body, 'plain'))

files = ['/content/drive/MyDrive/Colab Notebooks/Avocado_Land/data/sentence_emotion_231213_5pm.csv',]
        #  '/content/drive/MyDrive/Colab Notebooks/MNC-3rd-Contest/448048_sentencepiece.csv']

for file in files:
    with open(file, 'rb') as f:
        attach = MIMEApplication(f.read(),_subtype="csv")
        filename = os.path.basename(file)
        attach.add_header('Content-Disposition','attachment',filename=filename)
        msg.attach(attach)

# Send email
try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.login(gmail_user, gmail_pw)
    server.sendmail(mail_from, mail_to, msg.as_string())
    server.close()
    print("Successfully sent email")
except smtplib.SMTPException as e:
    print("Error: unable to send email", e)


In [ ]:
data.head()

,text_id,id,text,createdAt,updatedAt,default_emotion,gpt_emotion,user_input_emotion,birth_year,language,default_emotion_ko,posneg,klue_emo,klue_specific_emo,bareun,after_stopword,count_noun,keyword_emotions_scores,keyword_emotions_counts
1287,m#1696075442686#44b6c62f-a3f6-4022-982d-78ab7e...,P73XJUQdGKOMmVPdIcofddU8nOr2,몰라 진짜 나도 내 감정을 모르겠어 저저번주 주말부터 일주일동안 너가 맨날 연락해줘...,2023-09-30 12:04:02.686374,2023-10-02 16:33:29.394843,regret,regret,모르겠어,2009,KOREAN,후회하는,negative,negative,restless,"[나, 나, 감정, 저저번, 주, 주말, 일, 주일, 동안, 너, 나, 나, 너, ...","[감정, 저저번, 주, 주말, 주일, 이거, 상황, 서아, 상황, 승현이, 고등학교...","{'감정': 2, '저저번': 1, '주': 1, '주말': 1, '주일': 1, ...","[{'주변': 0.6594694206806387, '주': 0.65946942068...","[{'주변': 1, '주': 1, '표현': 1, '고다': 1, '연락': 1, ..."
1288,m#1696075544956#f6d1df90-cb59-4b02-8007-581555...,P73XJUQdGKOMmVPdIcofddU8nOr2,진짜 늦었지만 제로베이스원 데뷔 축하해 석매튜 사랑해,2023-09-30 12:05:44.956184,2023-10-02 16:33:29.376473,happy,happy,행복해요,2008,KOREAN,행복한,positive,positive,grateful,"[제로베이스원, 데뷔, 매튜]","[제로베이스원, 데뷔, 매튜]","{'제로베이스원': 1, '데뷔': 1, '매튜': 1}","[{'매튜': 0.4313020104174301}, {'데뷔': 0.28352649...","[{'매튜': 1}, {'데뷔': 1, '제로베이스원': 1}, {}]"
1517,m#1696075565296#3d2309c7-5fe1-4ccf-89de-4dfa3c...,TAyeOwri5NP96PLd4nA3IyGrIVs1,씨바꺼 진짜 족같네 이제야 우진이를 보낼 수 있을 줄 알았는데 뭐 우리집에 와서 2...,2023-09-30 12:06:05.296526,2023-09-30 12:06:05.296526,angry,angry,Angry,-,KOREAN,화난,negative,negative,rage,"[씨바, 진짜, 족, 우진이, 수, 줄, 우리, 집, 차, 나, 시험, 후, 번, ...","[족, 우진이, 집, 차, 시험, 후, 숙면, 이거]","{'족': 1, '우진이': 1, '집': 1, '차': 1, '시험': 1, '후...","[{}, {}, {'족': -0.9725436159884495, '후': -0.53...","[{}, {}, {'족': 1, '후': 1, '차': 1, '숙면': 1, '시험..."
1518,m#1696075679256#757102e8-94ec-4d50-9322-395228...,TAyeOwri5NP96PLd4nA3IyGrIVs1,그 꿈 이후로 왜 이렇게 애틋하고 여자가 너무젛고 나 여자 나 레즈,2023-09-30 12:07:59.256133,2023-10-07 16:00:42.966834,love,love,In Love,-,KOREAN,사랑하는,positive,negative,love,"[꿈, 이후, 여자, 너무젛, 고, 나, 여자, 나, 레즈]","[꿈, 이후, 여자, 너무젛, 고, 여자, 레즈]","{'꿈': 1, '이후': 1, '여자': 2, '너무젛': 1, '고': 1, '...","[{}, {'여자': -0.20748563160361722}, {'레즈': -1.0...","[{}, {'여자': 1}, {'레즈': 1, '꿈': 1, '여자': 1, '이후..."
1519,m#1696075741773#1793c953-4ce4-4732-a6f2-fb6386...,TAyeOwri5NP96PLd4nA3IyGrIVs1,집에 있는데도 집 가고 싶어 아니 쟤네들을 집보내고 싶어,2023-09-30 12:09:01.773689,2023-09-30 12:09:01.773689,lethargic,lethargic,Tired,2007,KOREAN,무기력한,negative,negative,annoyed,"[집, 집, 쟤]","[집, 집, 쟤]","{'집': 2, '쟤': 1}","[{}, {'집': -0.26172319508549}, {'집': -0.626373...","[{}, {'집': 1}, {'집': 2}]"


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
from torch.utils.data import DataLoader, Dataset

class KoreanSentimentDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
'거' in stopword_korean.values()